In [12]:
from hatchet import *
import json
from state import State
from preprocess import PreProcess
import numpy as np

In [13]:
dirname = '/Users/jarus/ucd/Research/Visualisation/projects/CallFlow/.callflow'
dataset = 'calc-pi'

In [21]:
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format

# Read entire dataset

In [22]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df

In [34]:
def write_gf(state, state_name, format_of_df):
    graph_literal = state.graph.to_literal(graph=state.graph, dataframe=state.df)
    print(graph_literal)
    graph_filepath = dirname + '/' + state_name + '/' + format_of_df + '_graph.json'
    with open(graph_filepath, 'w') as graphFile:
        json.dump(graph_literal, graphFile)

    # dump the filtered dataframe to csv.
    df_filepath = dirname + '/' + state_name + '/' + format_of_df + '_df.csv'
    state.df.to_csv(df_filepath)
    
    entire_df_filepath = dirname + '/' + state_name + '/' + 'entire' +'_df.csv'
    state.entire_df.to_csv(entire_df_filepath)

In [23]:
def read_gf(name):
    state = State()
    df_filepath = dirname + '/' + name +  '/filter_df.csv'
    entire_df_filepath = dirname + '/' + name + '/entire_df.csv'
    graph_filepath = dirname + '/' + name + '/filter_graph.json'
    entire_graph_filepath = dirname + '/' + name + '/entire_graph.json'   

    with open(graph_filepath, 'r') as graphFile:
        data = json.load(graphFile)

    state.gf = GraphFrame()
    state.gf.from_literal(data)

    with open(entire_graph_filepath, 'r') as entire_graphFile:
        entire_data = json.load(entire_graphFile)
            
    state.entire_gf = GraphFrame()
    state.entire_gf.from_literal(entire_data)

    state.df = pd.read_csv(df_filepath)
    state.entire_df = pd.read_csv(entire_df_filepath)

    state.graph = state.gf.graph
    state.entire_graph = state.entire_gf.graph

    # replace df['node'] from str to the Node object.
    state.df = replace_str_with_Node(state.df, state.graph)
    state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)

    return state

In [24]:
state = read_gf(dataset)

In [25]:
print(state.df)

    rank   time (inc)      time  nid  rank.1            file  line  \
0      0   999,238.00      0.00    0       0  <unknown file>     0   
1      1   999,390.00      0.00    0       1  <unknown file>     0   
2      2 1,000,306.00      0.00    0       2  <unknown file>     0   
3      3   999,308.00      0.00    0       3  <unknown file>     0   
4      0   999,238.00      0.00    1       0     ./src/cpi.c    19   
..   ...          ...       ...  ...     ...             ...   ...   
83     1    17,991.00 17,991.00   19       1        interp.c     0   
84     0    59,960.00 59,960.00   16       0        interp.c     0   
85     1    35,980.00 35,980.00   16       1        interp.c     0   
86     2    59,961.00 59,961.00   16       2        interp.c     0   
87     3    23,976.00 23,976.00   16       3        interp.c     0   

                 module            name            node type  n_index  \
0   libmonitor.so.0.0.0  <program root>  <program root>   PF        0   
1   libmonito

# Half-calc-pi dataset

In [26]:
state.df['time (inc)'] = state.df['time (inc)']*0.50
state.df['time'] = state.df['time']*0.50

In [27]:
state.entire_df['time (inc)'] = state.entire_df['time (inc)']*0.50
state.entire_df['time'] = state.entire_df['time']*0.50

In [28]:
print(state.df)

    rank  time (inc)      time  nid  rank.1            file  line  \
0      0  499,619.00      0.00    0       0  <unknown file>     0   
1      1  499,695.00      0.00    0       1  <unknown file>     0   
2      2  500,153.00      0.00    0       2  <unknown file>     0   
3      3  499,654.00      0.00    0       3  <unknown file>     0   
4      0  499,619.00      0.00    1       0     ./src/cpi.c    19   
..   ...         ...       ...  ...     ...             ...   ...   
83     1    8,995.50  8,995.50   19       1        interp.c     0   
84     0   29,980.00 29,980.00   16       0        interp.c     0   
85     1   17,990.00 17,990.00   16       1        interp.c     0   
86     2   29,980.50 29,980.50   16       2        interp.c     0   
87     3   11,988.00 11,988.00   16       3        interp.c     0   

                 module            name            node type  n_index  \
0   libmonitor.so.0.0.0  <program root>  <program root>   PF        0   
1   libmonitor.so.0.0.0  

In [33]:
write_gf(state, 'calc-pi-half', 'filter')

[{'name': '<program root>', 'children': [{'name': 'main', 'children': [{'name': '62:MPI_Finalize', 'children': [{'name': 'PMPI_Finalize', 'children': [{'name': '294:MPID_Finalize', 'children': [{'name': '162:MPIDI_CH3_Finalize', 'children': [{'name': '230:psm_dofinalize', 'children': [{'name': '36:<unknown procedure>', 'children': [{'name': '<unknown procedure>', 'children': [{'name': '<unknown procedure>', 'children': [{'name': '<unknown procedure>', 'children': [{'name': '<unknown file>:0', 'children': [], 'nid': 11, 'metrics': {}}, {'name': '<unknown procedure>', 'children': [{'name': '<unknown file>:0', 'children': [], 'nid': 13, 'metrics': {}}, {'name': '<unknown file>:0', 'children': [], 'nid': 14, 'metrics': {}}], 'nid': 12, 'metrics': {}}, {'name': '<unknown file>:0', 'children': [], 'nid': 15, 'metrics': {}}, {'name': '<unknown file>:0', 'children': [], 'nid': 16, 'metrics': {}}], 'nid': 10, 'metrics': {}}, {'name': 'interp.c:0', 'children': [], 'nid': 17, 'metrics': {}}, {'na

# 